In [11]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List

import pandas as pd
from ultralytics import YOLO


# =========================
# 你只需要改这里
# =========================
@dataclass
class EvalConfig:
    model_paths: List[str]
    data_yamls: List[str]
    split: str = "val"

    # 你要的 confidence 列表
    conf_list: List[float] = None

    # 评估参数（建议论文里固定）
    imgsz: int = 640
    batch: int = 16       # 默认16
    device: str = "0"     # "cpu" or "0"
    half: bool = True     # 默认True
    iou: float = 0.5      # 默认0.7
    max_det: int = 300    # 默认300

    # 输出目录
    out_dir: str = "./eval_tables"

    # 是否输出 per-class mAP (mAP50-95 per class)
    include_per_class: bool = True


CFG = EvalConfig(
    model_paths=[
        "/workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt",
    ],
    data_yamls=[
        "/workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml",
    ],
    split="val",
    conf_list=[0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.91, 0.92, 0.93],
    imgsz=640,
    batch=16,
    device="0",
    half=True,
    iou=0.5,
    max_det=300,
    out_dir="./eval_tables",
    include_per_class=True,
)


# =========================
# 小工具
# =========================
def safe_get(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)


def to_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


def f1(p: float | None, r: float | None) -> float | None:
    if p is None or r is None:
        return None
    if p + r == 0:
        return 0.0
    return 2 * p * r / (p + r)


def extract_row(metrics, model_path: str, data_yaml: str, split: str, conf: float) -> Dict[str, Any]:
    box = safe_get(metrics, "box", None)

    mp = to_float(safe_get(box, "mp", None))   # mean precision
    mr = to_float(safe_get(box, "mr", None))   # mean recall

    row: Dict[str, Any] = {
        "Model": Path(model_path).name,
        "DataYAML": Path(data_yaml).name,
        "Split": split,
        "Conf": conf,
        "Precision": mp,
        "Recall": mr,
        "F1": f1(mp, mr),
        "mAP50-95": to_float(safe_get(box, "map", None)),
        "mAP50": to_float(safe_get(box, "map50", None)),
        "mAP75": to_float(safe_get(box, "map75", None)),
    }

    if CFG.include_per_class:
        maps = safe_get(box, "maps", None)          # per-class mAP50-95
        names = safe_get(metrics, "names", None)    # id -> name
        if maps is not None and names is not None:
            if isinstance(names, dict):
                for cid, cname in names.items():
                    cid_int = int(cid)
                    if cid_int < len(maps):
                        row[f"mAP50-95[{cname}]"] = to_float(maps[cid_int])
            else:
                for cid, cname in enumerate(names):
                    if cid < len(maps):
                        row[f"mAP50-95[{cname}]"] = to_float(maps[cid])

    return row


def main():
    out_dir = Path(CFG.out_dir) / CFG.split
    out_dir.mkdir(parents=True, exist_ok=True)

    rows: List[Dict[str, Any]] = []

    for data_yaml in CFG.data_yamls:
        for model_path in CFG.model_paths:
            model = YOLO(model_path)

            for conf in CFG.conf_list:
                print(f"\n=== Evaluating ===")
                print(f"Model: {model_path}")
                print(f"Data : {data_yaml} | split={CFG.split} | conf={conf}")

                metrics = model.val(
                    data=data_yaml,
                    split=CFG.split,
                    imgsz=CFG.imgsz,
                    batch=CFG.batch,
                    device=CFG.device,
                    half=CFG.half,
                    conf=conf,          # <<< 关键：逐个conf跑
                    iou=CFG.iou,
                    max_det=CFG.max_det,
                    verbose=False,
                    plots=False,
                    workers=0,
                )

                rows.append(extract_row(metrics, model_path, data_yaml, CFG.split, conf))

    df = pd.DataFrame(rows)

    # 排序：更像论文表
    df = df.sort_values(["Model", "DataYAML", "Split", "Conf"]).reset_index(drop=True)

    # 数值列统一保留3位小数（论文常用）
    for c in df.columns:
        if c in ("Model", "DataYAML", "Split"):
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].round(3)

    # 导出（long-format：每行一个conf，很适合论文）
    csv_path = out_dir / "metrics_by_conf.csv"
    xlsx_path = out_dir / "metrics_by_conf.xlsx"
    tex_path = out_dir / "metrics_by_conf.tex"

    df.to_csv(csv_path, index=False)
    df.to_excel(xlsx_path, index=False)

    latex_df = df.copy()
    latex = latex_df.to_latex(index=False, escape=True)
    tex_path.write_text(latex, encoding="utf-8")

    # 额外：生成“宽表”（一眼对比每个conf的P/R/F1）
    # 对论文里一个模型一张表很友好
    wide_cols = ["Precision", "Recall", "F1", "mAP50-95", "mAP50", "mAP75"]
    wide = df.pivot_table(
        index=["Model", "DataYAML", "Split"],
        columns="Conf",
        values=wide_cols,
        aggfunc="first",
    )
    wide.columns = [f"{metric}@{conf}" for metric, conf in wide.columns]
    wide = wide.reset_index()

    wide_csv = out_dir / "metrics_by_conf_wide.csv"
    wide_xlsx = out_dir / "metrics_by_conf_wide.xlsx"
    wide_tex = out_dir / "metrics_by_conf_wide.tex"

    wide.to_csv(wide_csv, index=False)
    wide.to_excel(wide_xlsx, index=False)
    wide_tex.write_text(wide.to_latex(index=False, escape=True), encoding="utf-8")

    print("\n✅ Done.")
    print(f"- Long CSV : {csv_path.resolve()}")
    print(f"- Long XLSX: {xlsx_path.resolve()}")
    print(f"- Long TEX : {tex_path.resolve()}")
    print(f"- Wide CSV : {wide_csv.resolve()}")
    print(f"- Wide XLSX: {wide_xlsx.resolve()}")
    print(f"- Wide TEX : {wide_tex.resolve()}")


if __name__ == "__main__":
    main()



=== Evaluating ===
Model: /workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.5
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)


YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1976.7±725.2 MB/s, size: 28.6 KB)


val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:03<00:00, 13.39it/s]

                   all        759        785      0.957      0.929      0.963       0.84
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.9ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.6
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3120.1±539.0 MB/s, size: 59.8 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:03<00:00, 13.10it/s]

...

                   all        759        785      0.985      0.778      0.886      0.782
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.7ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.92
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2375.0±816.6 MB/s, size: 43.9 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:03<00:00, 14.77it/s]

                   all        759        785      0.996      0.603        0.8      0.714
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.7ms postprocess per image

=== Evaluating ===
Model: /workspace/_ty/02_models/best_models/04_swd_hbb/model_v2_4datasets_noAug_seed0_yolo11s_data_split_custom_8.pt
Data : /workspace/_ty/03_code/05_train_model/yolo/yaml/data_split_custom.yaml | split=val | conf=0.93
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2358.5±631.1 MB/s, size: 32.3 KB)



val: Scanning /workspace/_ty/03_code/05_train_model/yolo/data_custom/custom/val/labels.cache... 759 images, 0 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:02<00:00, 16.11it/s]

                   all        759        785          1      0.322      0.661      0.603
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.5ms postprocess per image

✅ Done.
- Long CSV : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.csv
- Long XLSX: /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.xlsx
- Long TEX : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf.tex
- Wide CSV : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.csv
- Wide XLSX: /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.xlsx
- Wide TEX : /workspace/_ty/03_code/05_train_model/yolo/eval_tables/val/metrics_by_conf_wide.tex
